# Synopsis-to-Vector

In [1]:
import numpy as np
import pandas as pd
import pickle
import math

## Load Pre-trained CBOW Parameters

In [2]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

## Get TF-IDF from synopsis.csv

### Load Cleaned Synopsis Data

In [3]:
synopsis = pd.read_csv('../clean_data/synopsis.csv')
synopsis.head()

,MAL_ID,Synopsis
0,1,solar leaving surface planet earth solar polic...
1,5,day life crew routine interrupted chasing targ...
2,6,head reason waste oppose entire cities fun tit...
3,7,individuals powers mind control robin craft us...
4,8,dark century people suffering rule manipulate ...


### Get Word Set from Synopsis Data

In [4]:
syns = synopsis.Synopsis.str.findall('\w+')

vocab = set()
for syn in syns:
    vocab.update(syn)

vocab = sorted(list(vocab))
vocab_size = len(vocab)

### Get TF(Term Frequency)

In [5]:
def get_tf(word, syn):
    # Term Frequency
    return syn.count(word)

    
tf = []
for syn in syns:
    tf_syn = []
    for word in vocab:
        tf_syn.append(get_tf(word, syn))
    tf.append(tf_syn)

tf = np.asarray(tf)
tf.shape

(6000, 5918)

### Get IDF(Inverse Document Frequency)

In [6]:
def get_idf(term):
    # Inverse Document Frequency
    _df = 0
    for syn in syns:
        _df += int(word in syn)
    return math.log(vocab_size/(_df))


idf = []
for word in vocab:
    idf.append(get_idf(word))

idf = np.asarray(idf)
idf.shape

(5918,)

### Get TF-IDF Table

In [7]:
tf_idf = pd.DataFrame(tf * idf, columns=vocab)

## Generate Sentence Vector of Synopsis

In [8]:
def get_word_vec(word):
    return word_vecs[word_to_id[word]]

def get_tf_idf(word, i):
    return tf_idf.loc[i, word]

def get_sent_vec(i):
    sent_vec = np.zeros(100, dtype=np.float16)

    for word in set(syns[i]):
        sent_vec += get_word_vec(word) * get_tf_idf(word, i)
        # sent_vec += get_word_vec(word)
    
    return sent_vec / len(set(syns[i]))

In [9]:
full_sent_vec = []
for i in range(len(syns)):
    full_sent_vec.append(get_sent_vec(i))
    
sent_vec_df = pd.DataFrame(full_sent_vec)
sent_vec_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.114929,-3.919922,0.509766,1.326172,2.685547,1.790039,-1.767578,-1.253906,-1.898438,1.072266,...,0.861816,-0.739746,-1.121094,-0.728516,1.063477,1.272461,-0.246826,1.380859,1.921875,2.222656
1,2.312500,-2.363281,-0.013573,2.707031,1.198242,3.175781,-1.357422,-2.603516,-1.961914,-0.565918,...,0.494629,-0.945801,-0.937988,-0.260742,2.085938,1.502930,-0.221436,0.645996,2.216797,0.961426
2,-0.225952,-2.599609,-0.183838,2.042969,1.673828,1.954102,-1.365234,-2.914062,-0.161377,0.051727,...,-1.434570,-1.021484,-2.539062,-2.451172,2.013672,1.943359,0.622559,1.706055,2.357422,1.231445
3,0.844238,-0.527832,0.271484,0.753418,1.575195,2.496094,-1.651367,-2.705078,-2.271484,1.470703,...,-0.367676,-1.790039,-1.804688,-2.474609,2.878906,0.426025,-0.582520,3.105469,0.837891,1.043945
4,1.521484,-2.652344,0.422852,1.815430,2.029297,1.694336,0.608887,-1.333984,-1.023438,0.937500,...,1.652344,-1.703125,-2.476562,-4.484375,1.402344,0.638672,0.058960,1.250000,2.455078,2.517578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.906250,-1.422852,1.900391,2.384766,1.668945,0.900879,-1.925781,-0.842773,-1.282227,1.340820,...,1.070312,-0.566406,-1.357422,-2.287109,0.418945,1.646484,0.828613,1.502930,2.164062,2.939453
5996,2.138672,-1.449219,0.351318,2.123047,0.429932,-0.171387,-0.983398,-2.195312,-2.029297,1.024414,...,-0.836914,-1.895508,-2.449219,-2.021484,1.226562,0.562500,-1.254883,2.990234,2.496094,1.022461
5997,1.149414,-2.382812,0.937988,0.430908,1.120117,2.529297,-1.702148,-1.220703,-1.709961,1.553711,...,0.711914,-0.082153,-1.972656,-0.666504,2.466797,0.973633,1.821289,-0.183350,2.941406,0.713379
5998,0.944824,-1.448242,-0.097656,0.824707,1.159180,1.099609,-0.375000,-2.132812,-0.811035,2.246094,...,1.915039,0.847168,-1.115234,-1.065430,1.032227,1.030273,-0.781250,0.202026,2.701172,1.934570


In [10]:
sent_vec_df.insert(0, 'MAL_ID', synopsis.MAL_ID)
sent_vec_df

,MAL_ID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,-0.114929,-3.919922,0.509766,1.326172,2.685547,1.790039,-1.767578,-1.253906,-1.898438,...,0.861816,-0.739746,-1.121094,-0.728516,1.063477,1.272461,-0.246826,1.380859,1.921875,2.222656
1,5,2.312500,-2.363281,-0.013573,2.707031,1.198242,3.175781,-1.357422,-2.603516,-1.961914,...,0.494629,-0.945801,-0.937988,-0.260742,2.085938,1.502930,-0.221436,0.645996,2.216797,0.961426
2,6,-0.225952,-2.599609,-0.183838,2.042969,1.673828,1.954102,-1.365234,-2.914062,-0.161377,...,-1.434570,-1.021484,-2.539062,-2.451172,2.013672,1.943359,0.622559,1.706055,2.357422,1.231445
3,7,0.844238,-0.527832,0.271484,0.753418,1.575195,2.496094,-1.651367,-2.705078,-2.271484,...,-0.367676,-1.790039,-1.804688,-2.474609,2.878906,0.426025,-0.582520,3.105469,0.837891,1.043945
4,8,1.521484,-2.652344,0.422852,1.815430,2.029297,1.694336,0.608887,-1.333984,-1.023438,...,1.652344,-1.703125,-2.476562,-4.484375,1.402344,0.638672,0.058960,1.250000,2.455078,2.517578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,48438,0.906250,-1.422852,1.900391,2.384766,1.668945,0.900879,-1.925781,-0.842773,-1.282227,...,1.070312,-0.566406,-1.357422,-2.287109,0.418945,1.646484,0.828613,1.502930,2.164062,2.939453
5996,48466,2.138672,-1.449219,0.351318,2.123047,0.429932,-0.171387,-0.983398,-2.195312,-2.029297,...,-0.836914,-1.895508,-2.449219,-2.021484,1.226562,0.562500,-1.254883,2.990234,2.496094,1.022461
5997,48470,1.149414,-2.382812,0.937988,0.430908,1.120117,2.529297,-1.702148,-1.220703,-1.709961,...,0.711914,-0.082153,-1.972656,-0.666504,2.466797,0.973633,1.821289,-0.183350,2.941406,0.713379
5998,48483,0.944824,-1.448242,-0.097656,0.824707,1.159180,1.099609,-0.375000,-2.132812,-0.811035,...,1.915039,0.847168,-1.115234,-1.065430,1.032227,1.030273,-0.781250,0.202026,2.701172,1.934570


In [11]:
sent_vec_df.to_csv('synopsis_vector.csv', index=False)